In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgbm
#from xgboost import XGBRegressor
#import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc

In [2]:
sys.path.append(os.path.realpath('..'))

In [3]:
train_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\train.csv")
test_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\test.csv")
sample_sub = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\sample_submission.csv")

In [4]:
columns_to_vectorize = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for vector_target in columns_to_vectorize:
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

In [5]:
features = ['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'gender_v', 'ever_married_v', 'work_type_v', 'Residence_type_v', 'smoking_status_v']
target = ['stroke']

In [6]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [7]:
def lgbm_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 100, 1000), 
        'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        'n_estimators': trial.suggest_int('n_estimators', 10, 100000), 
    } 
    
    clf = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    
    clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)
    preds = clf.predict(X_valid)

    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    lgbm_ranks[roc_auc] = clf

    return roc_auc

In [8]:
lgbm_ranks = {}

train_time = 1 * 60 * 5
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='LGBMRegressor')
study.optimize(lgbm_objective, timeout=train_time)

[I 2023-01-16 14:44:02,272] A new study created in memory with name: LGBMRegressor
D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
[I 2023-01-16 14:44:02,451] Trial 0 finished with value: 0.8747899629391921 and parameters: {'learning_rate': 0.059545119507

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[27]	valid_0's auc: 0.87479


D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


Training until validation scores don't improve for 100 rounds


[I 2023-01-16 14:44:03,116] Trial 1 finished with value: 0.8738774171519547 and parameters: {'learning_rate': 0.02315841927963849, 'max_depth': 907, 'num_leaves': 939, 'colsample_bytree': 0.6048134670031027, 'subsample': 0.48357199779454807, 'subsample_freq': 9, 'min_child_samples': 10, 'reg_lambda': 8, 'n_estimators': 87331}. Best is trial 0 with value: 0.8747899629391921.


...

Early stopping, best iteration is:
[18]	valid_0's auc: 0.882482
Training until validation scores don't improve for 100 rounds


D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
[I 2023-01-16 14:49:02,131] Trial 929 finished with value: 0.8806029806237232 and parameters: {'learning_rate': 0.001225487038971506, 'max_depth': 746, 'num_leaves': 6777, 'colsample_bytree': 0.8954314155906

Early stopping, best iteration is:
[17]	valid_0's auc: 0.880603
Training until validation scores don't improve for 100 rounds


D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
[I 2023-01-16 14:49:02,534] Trial 930 finished with value: 0.8877925618426556 and parameters: {'learning_rate': 0.016017905149661994, 'max_depth': 775, 'num_leaves': 2260, 'colsample_bytree': 0.8786025630196

Early stopping, best iteration is:
[120]	valid_0's auc: 0.887793


In [9]:
LGBM_ranks_bytes = pickle.dumps(lgbm_ranks)
with open("rendered_data/lgbm_clsfs_bytes.hex", "wb") as binary_file:
    binary_file.write(LGBM_ranks_bytes)

In [16]:
percent = math.ceil(len(lgbm_ranks.keys()) * .1)
top = sorted(list(lgbm_ranks.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    train_preds.append(lgbm_ranks[key].predict(X))
    test_preds.append(lgbm_ranks[key].predict(X_test))
    
train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|█████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 108.94it/s]


In [17]:
LGBM_train_submission = pd.DataFrame(data={'id': train_df.id, 'stroke': train_final_preds})
LGBM_train_submission.to_csv(fr'rendered_data/LGBM_train_submission.csv', index=False)

LGBM_test_submission = pd.DataFrame(data={'id': test_df.id, 'stroke': test_final_preds})
LGBM_test_submission.to_csv(fr'rendered_data/LGBM_test_submission.csv', index=False)